### Prepare Repo

In [1]:
import os
os.environ["TORCH_HOME"]="/data/chuak"

In [2]:
!pip install git+https://github.com/ChuakBlurk/vidaug
!pip install omegaconf==2.1.1
!pip install hydra-core==1.1.1
!pip install -U numpy==1.23.5
!apt-get update && apt-get install -y python3-opencv
!pip install opencv-python
!pip install scikit-image 
!pip install transformers
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install wandb
!pip install scikit-learn
!pip install facenet-pytorch

  Cloning https://github.com/ChuakBlurk/vidaug to /tmp/pip-req-build-0zx6qul2
  Running command git clone -q https://github.com/ChuakBlurk/vidaug /tmp/pip-req-build-0zx6qul2
  Resolved https://github.com/ChuakBlurk/vidaug to commit f074c68d127cb6fadd2d7985e1bc3f633104d45d
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
apt-get: Command not found.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You

In [3]:
!git clone https://github.com/ChuakBlurk/av_hubert.git
!pip install facenet-pytorch
%cd av_hubert
!git submodule init
!git submodule update
!pip install scipy
!pip install sentencepiece
!pip install python_speech_features
!pip install scikit-video

%cd fairseq
!pip install ./

fatal: destination path 'av_hubert' already exists and is not an empty directory.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/av_hubert


/data/chuak/mmser/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/chuak/mmser/bin/python-csd-3.10 -m pip install --upgrade pip' command.
/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/av_hubert/fairseq
Processing /data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/av_hubert/fairseq
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at 

In [2]:
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
import cv2
import tempfile
import torch
from transformers import Trainer, TrainingArguments
import sys
sys.path.append("/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert")
%cd /home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/
import utils as avhubert_utils
from argparse import Namespace
from IPython.display import HTML
import numpy as np
import sys
print(sys.version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import wandb
from torch.utils.data import Dataset, Subset
import os
import datetime

[Errno 2] No such file or directory: '/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/'
/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video
3.10.0 (default, Oct  6 2021, 10:19:31) [GCC 8.3.1 20190311 (Red Hat 8.3.1-3)]


/data/chuak/mmser/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [3]:
wandb.init()

wandb: Currently logged in as: mmser. Use `wandb login --relogin` to force relogin


### Download Model

In [6]:
os.makedirs("/data/chuak/mmser/check_pts/")
!wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/clean-pretrain/base_vox_iter4.pt -O /data/chuak/mmser/check_pts/avhubert.pt

FileExistsError: [Errno 17] File exists: '/data/chuak/mmser/check_pts/'

### Build Model Pipeline

### Load Dataset

In [4]:
from avhubert_ds import AVHUBERTDataset
mmser_ds = torch.load("/data/chuak/mmser/data/avhubert_ds2.pt")
mmser_ds.video_path = "/data/chuak/mmser/data/roi/"
mmser_ds.cached = False

In [5]:
mmser_ds.cached = False
mmser_ds.__cache__()

 45%|████████████████████████▊                              | 2492/5531 [03:01<03:41, 13.70it/s]


KeyboardInterrupt: 

### Define the model

In [ ]:
from avhubert_classifier import AVHUBERTClassifier
user_dir = "/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "/data/chuak/mmser/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")
avhubert_classifier = AVHUBERTClassifier(model, 768, 256)

In [ ]:
avhubert_classifier(**mmser_ds[:3])
avhubert_res = avhubert_classifier.encoder.feature_extractor_video.resnet
video_input = mmser_ds[:3]["video"]
avhubert_res_output = avhubert_res(video_input)
print(video_input.shape)
print(avhubert_res_output.shape)

In [ ]:
import torch.nn as nn
class FaceNetBlock(nn.Module):
    
    def __init__(self, in_channels, facenet):
        super(FaceNetBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = 3
        self.facenet = facenet
    def forward(self, x):
        x = self.facenet(x)
        return x

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
facenet_res = InceptionResnetV1(pretrained='vggface2')
faceNetBlock = FaceNetBlock(64, facenet_res)
avhubert_classifier.encoder.feature_extractor_video.resnet.trunk = faceNetBlock
avhubert_classifier.encoder.feature_extractor_video.resnet.frontend3D =nn.Conv3d(1, 3, 
                                      kernel_size=(5, 7, 7), 
                                      stride=(1, 1, 1), 
                                      padding=(2, 3, 3), bias=False)

In [ ]:
avhubert_classifier(**mmser_ds[:3])

### Build Train Test DS

In [7]:
meta_df_ = mmser_ds.df_
mmser_ds.df_["bigsess"] = mmser_ds.df_["session"].apply(lambda x: x[:-1])
sess_dict = mmser_ds.df_.groupby("bigsess").groups
all_indices = set(mmser_ds.df_.index.tolist())

### Data Augmentation

In [8]:
from torch.utils.data import Dataset, Subset
import os
import pandas as pd
import numpy as np
import torch 
import torch.nn.functional as F
import sys
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import vidaug.augmentors as va
import random


class VidaugDataset(Dataset):
    
    def collate(self, audio, video, max_size=500):
        padded_audio = pad_sequence([torch.tensor(a.squeeze()) for a in audio]+[torch.empty(500, 104)], batch_first=True)[:-1]
        padded_video = pad_sequence([v.squeeze().clone().detach() for v in video]+[torch.empty(500,88,88)], batch_first=True)[:-1, np.newaxis, : ,:,:]
        mask = torch.zeros_like(padded_audio)
        mask[padded_audio != 0] = 1
        return padded_audio, padded_video, mask
    
    
    def __init__(self, audio_feats_list, 
                 video_feats_list, 
                 text_list, 
                 labels_list, aug_prob=0.3):

        self.label_smp_dict = {}
        for idx, label in enumerate(labels_list):
            if label not in self.label_smp_dict:
                self.label_smp_dict[label] = []
            self.label_smp_dict[label].append({
                "audio": audio_feats_list[idx],
                "video": video_feats_list[idx],
                "text": text_list[idx],
                "label": labels_list[idx],
            })
        self.aug = False
        self.aug_len = 0
        self.sometimes = lambda aug: va.Sometimes(aug_prob, aug) # Used to apply augmentor with 50% probability
        
        self.transform_list = [
            self.sometimes(va.Salt()),
            self.sometimes(va.Pepper()),
            self.sometimes(va.RandomShear(0.2, 0.2)),
            self.sometimes(va.HorizontalFlip()),
            self.sometimes(va.VerticalFlip()),
            self.sometimes(va.RandomRotate(30)),
            self.sometimes(va.GaussianBlur(0.8)),
            self.sometimes(va.ElasticTransformation(0.2,0.2)),
            self.sometimes(va.PiecewiseAffineTransform(20,10,0.5)),
        ]
        

    def __upsample__(self, origin_smp_list, smp_length, k=None):
        smpled_list = origin_smp_list
        for idx in tqdm(range(smp_length-len(origin_smp_list))):
            seq = va.Sequential(random.choices(self.transform_list, k=k))
            smp = random.choice(origin_smp_list)
            vid = smp["video"].squeeze()
            vid = [cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB).astype(np.uint8) for frame in vid]
            vid = np.stack(vid)
            video_aug = seq(vid)
            video_aug = [cv2.cvtColor(frame.astype(np.float32), cv2.COLOR_RGB2GRAY).astype(np.uint8) for frame in video_aug]
            video_aug = np.stack(video_aug)
            video_aug = video_aug[np.newaxis, np.newaxis]
            
            smpled_list.append({
                "video": video_aug,
                "audio": smp['audio'],
                "text": smp['text'],
                "label": smp['label'],
            })
        return smpled_list
            
        
    
    def __aug__(self, niters=2, nchoice=2, isaug=True):
        self.aug_label_smp_dict = {}
        self.aug_smps = []
        if isaug:
            print([len(v) for k,v in self.label_smp_dict.items()])
            smp_size = max([len(v) for k,v in self.label_smp_dict.items()])*(niters+1)
            for k, v in self.label_smp_dict.items():
                print(smp_size, len(v))
                upsmped = self.__upsample__(v, smp_size, nchoice)
                self.aug_label_smp_dict[k] = upsmped
                self.aug_smps += upsmped
                
            self.aug = True
            print([len(v) for k,v in self.aug_label_smp_dict.items()])
        else:
            print([len(v) for k,v in self.label_smp_dict.items()])
            for k, v in self.label_smp_dict.items():
                self.aug_label_smp_dict[k] = v
                self.aug_smps += v
            self.aug = True
            print([len(v) for k,v in self.aug_label_smp_dict.items()])
            
    
    def __len__(self):
        return len(self.aug_smps)
    
    def __getitem__(self, idx):
        audio_feats = self.aug_smps[idx]["audio"]
        video_feats = self.aug_smps[idx]["video"]/255
        padded_audio, padded_video, padding_mask = self.collate([audio_feats], [torch.tensor(video_feats)])
        return {
            "padding_mask": padding_mask[0][:500, :].float(),
            "audio": padded_audio[0][:500, :].T.float(),
            "video": padded_video[0][:, :500, :, :].float(),
            "text": self.aug_smps[idx]["text"],
            "labels": self.aug_smps[idx]["label"]
        }

# class VidaugDataset(Dataset):
    
#     def collate(self, audio, video, max_size=500):
#         padded_audio = pad_sequence([torch.tensor(a.squeeze()) for a in audio]+[torch.empty(500, 104)], batch_first=True)[:-1]
#         padded_video = pad_sequence([v.squeeze().clone().detach() for v in video]+[torch.empty(500,88,88)], batch_first=True)[:-1, np.newaxis, : ,:,:]
#         mask = torch.zeros_like(padded_audio)
#         mask[padded_audio != 0] = 1
#         return padded_audio, padded_video, mask
    
    
#     def __init__(self, audio_feats_list, 
#                  video_feats_list, 
#                  text_list, 
#                  labels_list):
#         self.audio_feats_list = audio_feats_list
#         self.video_feats_list = video_feats_list
#         self.text_list = text_list
#         self.labels_list = labels_list
    
#         print(len(self.audio_feats_list))
#         print(len(self.video_feats_list))
#         print(len(self.text_list))
#         print(len(self.labels_list))
        
#         self.origin_len = len(self.labels_list)
#         self.aug_len = 0

    
    
#     def __aug__(self, niters=2, aug_prob=0.3, k=None):
#         sometimes = lambda aug: va.Sometimes(aug_prob, aug) # Used to apply augmentor with 50% probability
        
#         transform_list = [
#             # sometimes(va.InvertColor()),
#             sometimes(va.Salt()),
#             sometimes(va.Pepper()),
#             sometimes(va.RandomShear(0.2, 0.2)),
#             sometimes(va.HorizontalFlip()),
#             sometimes(va.VerticalFlip()),
#             sometimes(va.RandomRotate(30)),
#             sometimes(va.GaussianBlur(0.8)),
#             sometimes(va.ElasticTransformation(0.2,0.2)),
#             sometimes(va.PiecewiseAffineTransform(20,10,0.5)),
#         ]
        
        
#         self.aug_audio_feats_list = []
#         self.aug_video_feats_list = []
#         self.aug_text_list = []
#         self.aug_labels_list = []
        
#         for smp_id in tqdm(range(len(self.video_feats_list))):
#             for i in range(niters):
#                 if k is None:
#                     seq = va.Sequential(random.choices(transform_list, 
#                                                    k=random.choice(range(len(transform_list)))))        
#                 else:
#                     seq = va.Sequential(random.choices(transform_list, 
#                                                    k=k))
#                 vid = self.video_feats_list[smp_id].squeeze()
#                 # change to color 
#                 vid = [cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB).astype(np.uint8) for frame in vid]
#                 vid = np.stack(vid)
#                 video_aug = seq(vid)
#                 # print(video_aug[0].shape)
#                 video_aug = [cv2.cvtColor(frame.astype(np.float32), cv2.COLOR_RGB2GRAY).astype(np.uint8) for frame in video_aug]
#                 video_aug = np.stack(video_aug)
                
#                 # transform = avhubert_utils.Compose([
#                 #   avhubert_utils.Normalize(0.0, 255.0),
#                 #   avhubert_utils.CenterCrop((88, 88)),
#                 #   avhubert_utils.Normalize(0.421, 0.165)])
#                 # video_aug = transform(video_aug)[np.newaxis, np.newaxis]

#                 video_aug = (video_aug/255)[np.newaxis, np.newaxis]

                
#                 video_aug = torch.tensor(video_aug)
#                 self.aug_audio_feats_list.append(self.audio_feats_list[smp_id])
#                 self.aug_video_feats_list.append(video_aug)
#                 self.aug_text_list.append(self.text_list[smp_id])
#                 self.aug_labels_list.append(self.labels_list[smp_id])
                
#         self.aug_len = len(self.aug_labels_list)
                
        
#     def __len__(self):
#         return self.origin_len + self.aug_len
    
#     def __getitem__(self, idx):
#         if idx < self.origin_len:
#             audio_feats = self.audio_feats_list[idx]
#             video_feats = self.video_feats_list[idx]
#             padded_audio, padded_video, padding_mask = self.collate([audio_feats], [torch.tensor(video_feats)])
#             return {
#                 "padding_mask": padding_mask[0][:500, :].float(),
#                 "audio": padded_audio[0][:500, :].T.float(),
#                 "video": padded_video[0][:, :500, :, :].float(),
#                 "text": self.text_list[idx],
#                 "labels": self.labels_list[idx]
#             }
#         else:
#             idx = idx - self.origin_len
#             audio_feats = self.aug_audio_feats_list[idx]
#             video_feats = self.aug_video_feats_list[idx]
#             padded_audio, padded_video, padding_mask = self.collate([audio_feats], [torch.tensor(video_feats)])
#             return {
#                 "padding_mask": padding_mask[0][:500, :].float(),
#                 "audio": padded_audio[0][:500, :].T.float(),
#                 "video": padded_video[0][:, :500, :, :].float(),
#                 "text": self.aug_text_list[idx],
#                 "labels": self.aug_labels_list[idx]
#             }

In [9]:
audio_feats_list = mmser_ds.audio_feats_list
video_feats_list = mmser_ds.video_feats_list
text_list = list(meta_df_["transcript"])
labels_list = list(meta_df_["emotion_id"])

del mmser_ds

In [10]:
val_indices = sess_dict['Ses03']
test_indices = sess_dict['Ses04']
train_indices = list(all_indices-set(val_indices)-set(test_indices))

In [11]:
train_ds = VidaugDataset(
    [item.detach().numpy() for i, item in enumerate(audio_feats_list) if i in train_indices],
    [item.detach().numpy() for i, item in enumerate(video_feats_list) if i in train_indices],
    [item for i, item in enumerate(text_list) if i in train_indices],
    [item for i, item in enumerate(labels_list) if i in train_indices]
)

val_ds = VidaugDataset(
    [item.detach().numpy() for i, item in enumerate(audio_feats_list) if i in val_indices],
    [item.detach().numpy() for i, item in enumerate(video_feats_list) if i in val_indices],
    [item for i, item in enumerate(text_list) if i in val_indices],
    [item for i, item in enumerate(labels_list) if i in val_indices]
)

test_ds = VidaugDataset(
    [item.detach().numpy() for i, item in enumerate(audio_feats_list) if i in test_indices],
    [item.detach().numpy() for i, item in enumerate(video_feats_list) if i in test_indices],
    [item for i, item in enumerate(text_list) if i in test_indices],
    [item for i, item in enumerate(labels_list) if i in test_indices]
)

In [12]:
set(labels_list)

{0.0, 1.0, 2.0, 3.0}

In [13]:
train_ds.__aug__(2)

[1130, 536, 636, 1047]
3390 1130


100%|████████████████████████████████████████████████████| 2260/2260 [01:35<00:00, 23.67it/s]


3390 536


100%|████████████████████████████████████████████████████| 2854/2854 [02:26<00:00, 19.44it/s]


3390 636


100%|████████████████████████████████████████████████████| 2754/2754 [02:53<00:00, 15.89it/s]


3390 1047


100%|████████████████████████████████████████████████████| 2343/2343 [01:46<00:00, 22.04it/s]

[3390, 3390, 3390, 3390]


In [14]:
val_ds.__aug__(isaug=False)
test_ds.__aug__(isaug=False)

[320, 286, 305, 240]
[320, 286, 305, 240]
[327, 143, 303, 258]
[327, 143, 303, 258]


In [15]:
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))
train_ds[0]["video"].shape

13560
1151
1031


torch.Size([1, 500, 88, 88])

### Run Pipeline

API: 2999b8f99f0f62b4f64c48a1c8be9a16945183e9

In [16]:
from avhubert_classifier import AVHUBERTClassifier
user_dir = "/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "/data/chuak/mmser/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")

/data/chuak/mmser/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Checkpoint: pre-trained w/o fine-tuning


In [17]:
import json
# avhubert_classifier = AVHUBERTClassifier(model, 768, 256)
avhubert_classifier = AVHUBERTClassifier(model, 768, 64)
for param in avhubert_classifier.parameters():
    param.requires_grad = True

wandb.init()
train_set = train_ds
val_set = val_ds
test_set = test_ds


wandb: Currently logged in as: mmser. Use `wandb login --relogin` to force relogin


### Change resnet weight

In [18]:
from facenet_pytorch import MTCNN, InceptionResnetV1
# avhubert_classifier.encoder.feature_extractor_video.resnet = InceptionResnetV1(pretrained='vggface2')
avhubert_classifier = avhubert_classifier.to(device)
avhubert_classifier = AVHUBERTClassifier(model, 768, 32)

facenet_res = InceptionResnetV1(pretrained='vggface2')
faceNetBlock = facenet_res
avhubert_classifier.encoder.feature_extractor_video.resnet.trunk = faceNetBlock
avhubert_classifier.encoder.feature_extractor_video.resnet.frontend3D =nn.Conv3d(1, 3, 
                                      kernel_size=(5, 7, 7), 
                                      stride=(1, 1, 1), 
                                      padding=(2, 3, 3), bias=False)

In [19]:
output_dir=os.path.join("check_pts", "AVHUBERT", datetime.datetime.now().date().strftime(format="%Y-%m-%d"))

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=2
training_args.per_device_eval_batch_size=2
training_args.logging_steps = int(1000/training_args.per_device_train_batch_size/8)
training_args.eval_steps = int(1000/training_args.per_device_train_batch_size/8)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=5e-4
training_args.num_train_epochs=7
training_args.metric_for_best_model = 'accuracy'

In [20]:
from avhubert_trainer import CustomTrainer , compute_metrics
from transformers import EarlyStoppingCallback, TrainerCallback, TrainerState

trainer = CustomTrainer(
    model=avhubert_classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


##### Gradual Freezing

In [21]:
class FreezingCallback(TrainerCallback):
    
    def __init__(self, freeze_encoder_epochs: int):
        self.freeze_encoder_epochs = freeze_encoder_epochs

    def on_epoch_begin(self, args, state, control, **kwargs):
        print(state.epoch, self.freeze_encoder_epochs)
        model = kwargs["model"]
        if state.epoch >= self.freeze_encoder_epochs:
            print("="*10, "Freezing", "="*10)
            for param in model.encoder.feature_extractor_video.parameters():
                param.requires_grad = False

    def on_save(self, args, state, control, **kwargs):
        model = kwargs["model"]
        for name, param in model.named_parameters():
            param.requires_grad = True

In [22]:
training_args.device

device(type='cuda', index=0)

In [23]:
# freezing_callback = FreezingCallback(3)
# trainer.add_callback(freezing_callback)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wa,Ua,F1,Accuracy
62,1.399800,1.430543,0.248480,0.250000,0.098908,0.248480
124,1.398200,1.413233,0.248480,0.250000,0.098908,0.248480


/data/chuak/mmser/src/multi_modal_ser/finetune_encoder/audio_video/avhubert_trainer.py:34: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_f1 = load_metric("f1")


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(faceNetBlock)

In [90]:
count_parameters(avhubert_classifier.encoder.feature_extractor_video.resnet)

11186688

In [91]:
del trainer

In [ ]:
val_preds = trainer.predict(val_set)

In [ ]:
val_preds

In [ ]:
import pandas as pd
pred_labels = val_preds.predictions.argmax(axis=1)
true_labels = val_preds.label_ids

In [ ]:
print(pred_labels[10:15])
print(true_labels[10:15])

In [ ]:
pd.Series(true_labels).value_counts()

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(true_labels, pred_labels, average=None)

In [ ]:
loss = nn.CrossEntropyLoss()
output = loss(torch.tensor(val_preds.predictions), torch.tensor(true_labels).long())

In [ ]:
output

In [ ]:
eval_result = trainer.evaluate()
test_result = trainer.predict(test_set).metrics

In [ ]:
test_result